In [ ]:
# Before we start, load our environment
%matplotlib inline
import os
import re
import sys
import argparse
import csv
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

# Read the organisms metadata.

We have a file called `patric_metadata_20180526_isolation_host_env.tsv` that we want to parse. It has the columns `['genome_id', 'isolation_source', 'host_name', 'Environment]` and the main data that we want to classify against are host_name and Environment. Isolation_source is the original classification in PATRIC. Host_name is sparse - not everything has a host name.

In [ ]:
def read_classifications(cf):
    """
    Read the classifications file. We expect that this has 4 tab-separated values.
    :param cf: the classifications file
    :return: two dicts of the hostname and environment. Recall, not everything has a hostname
    """
    hostname = {}
    environment = {}
    with open(cf, 'r') as f:
        for l in f:
            if l.startswith('genome_id'):
                continue
            p=l.strip().split("\t")
            if p[2]:
                hostname["PATRIC|{}".format(p[0])] = p[2]
            if p[3]:
                environment["PATRIC|{}".format(p[0])] = p[3]
    return hostname, environment

hostname, environment = read_classifications('patric_data/patric_metadata_isolation_host_env.tsv')                

# Read the focus output tsv file

We have a directory of focus output files that are compressed, and we parse those into a single .tsv file that has one genome per line and the columns are the metagenomes.

We parse that into a datastructure automatically. Note that telling pandas we will use the first row as header (header = 0) and the first column as the index (index_col = 0) are key to later pandas magic.

Note that our original file [metagenome_counts_20180625.tsv.gz](metagenome_counts_20180625.tsv.gz) is super huge - (28,705 by 18,920) and so we don't want to use that here. I wrote a [small script](create_dev_dataset.py) to pull out 100 metagenomes and all genomes that are not zero in those metagenomes. It chooses the 99 at random and you can also change the number of metagenomes selected. In my example, [dev_counts.tsv](dev_counts.tsv) the data set is (99 x 3,029) and so much, much faster!

In [ ]:
def read_focus_output(fof):
    """
    :param fof: focus output file
    :return: a pandas data frame with the data
    """
    
    df = pd.read_csv(fof, sep="\t", header=0, index_col=0,)
    return df


# NOTE: The following file is (28,705 x 18,920 and so takes a long time to read!)
# focus = read_focus_output("metagenome_counts.tsv")
# this file is 3,029 genomes x 99 metagenomes
focus = read_focus_output('example_data/dev_counts_sel.tsv')
focus.head(10)

In [ ]:
focus['environment'] = pd.Series(environment)
focus['hostname'] = pd.Series(hostname)

In [ ]:
focus.head(10)

In [ ]:
# write this to a csv so I can get help!
focus.to_csv("/home/redwards/Desktop/metagenomes_genomes.tsv", sep="\t")

# NOTE: THIS IS WRONG!

We want to classify the metagenomes by the genomes, not vice versa! We're doing it wrong here. SEE BELOW

# Extract the features that we want to use in our random forest

Now that we have merged everything, we extract the column names of the features that we want to use in our random forest. This just creates an index of the species that we have. Note that if you use genus you will need to change the 492 to something else!

In [ ]:
features = focus.columns[:-2]

# Create test and training sets.

For our classifier, we are going to use some part of the data to train the random forest classifier, and some part of the data to test it. We are going to make a new column that says whether it is testing or training, and populate it so that 75% of the data is training and 25% of the data is for testing. You can change those variables here. 

Note that in this example we are using the same data sets for training and testing. Before publication we should come up with sets of examplar metagenomes from each environment that we have manually curated and use those to train our classifiers.

In [ ]:
focus['is_train'] = np.random.uniform(0, 1, len(focus)) <= .75
train, test = focus[focus['is_train']==True], focus[focus['is_train']==False]
print("Data: {}\nTraining: {}\nTesting: {}".format(focus.shape, train.shape, test.shape))

# Create a factorized list of environments

The random forest requires the environments to just be a list of integers rather than labels, and so we use factorize to split out the labels and the indexes. This will be the input to our random forest classifier. Note that we are just using the training set here, not the whole data frame.

In [ ]:
envfactors, labels = pd.factorize(train['environment'])
labels

# Start a random classifier

I leave most of this at the default (which is to use Gini as the measure of quality), and to bootstrap the trees.

In [ ]:
clf = RandomForestClassifier(n_jobs=-1)

# Train the random forest

Now we train our random forest on the training data using just the features that we are looking for and with our factorized list of diagnoses

In [ ]:
clf.fit(train[features], envfactors)

# Test our classifier using the test data set

This makes predictions for each of the test data sets. For each sample we get a number depending on which environment we predict it is from.

Then we convert those predictions to the appropriate labels. We now have an array of predictions the same length as our test data set.

In [ ]:
clf.predict(test[features])
predictions=labels[clf.predict(test[features])]
predictions[0:10]

And then we can compare those predictions to the test data set. As predicted we confuse 'human gut' with 'waste water'. Looks like we need a confusion matrix!

In [ ]:
test.environment[0:10]

## Why is this wrong?

In this random forest, we are predicting the source of the bacteria based on the environments that they come from ... i.e. we are around the wrong way.

We want to predict the source of the environments based on the bacteria that they contain.


# Recreate the data and rotate it.

Here, I just call the methods from above to get the data.

In [ ]:
hostname, environment = read_classifications('patric_data/patric_metadata_isolation_host_env.tsv')                
focus = read_focus_output('example_data/dev_counts.tsv')
focus.head()

In [ ]:
ft = focus.T
ft.head()